In [ ]:
import requests
import pandas as pd
import numpy as np
from skimage.draw import line_aa  # Importing line_aa for drawing lines

# Function to rescale strokes coordinates from [0, 255] to [0, 27]
def rescale_strokes(strokes):
    return [[np.array(x) / 10, np.array(y) / 10] for x, y in strokes]

# Function to convert strokes to pixel image
def strokes_to_pixels(strokes, image_shape=(28, 28)):
    # Create an empty pixel grid
    pixels = np.zeros(image_shape)

    # Iterate over each stroke
    for stroke in strokes:
        # Interpolate the stroke onto the pixel grid
        for i in range(len(stroke[0]) - 1):
            x0, y0 = int(stroke[0][i]), int(stroke[1][i])
            x1, y1 = int(stroke[0][i + 1]), int(stroke[1][i + 1])

            # Map stroke coordinates to pixel grid
            x0 = max(0, min(image_shape[1] - 1, x0))
            y0 = max(0, min(image_shape[0] - 1, y0))
            x1 = max(0, min(image_shape[1] - 1, x1))
            y1 = max(0, min(image_shape[0] - 1, y1))

            rr, cc, val = line_aa(y0, x0, y1, x1)
            pixels[rr, cc] = np.maximum(pixels[rr, cc], val)  # Update pixel values

    return pixels

# URLs for the drawings
urls = [
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/The%20Eiffel%20Tower.ndjson",      #effiel tower
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/The%20Great%20Wall%20of%20China.ndjson",    #great wall of china
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/The%20Mona%20Lisa.ndjson",    #mona lisa
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/aircraft%20carrier.ndjson",   #aircraft carrier
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/airplane.ndjson",     #aircraft
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/alarm%20clock.ndjson",  #alarm clock
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/ambulance.ndjson",     #ambulance
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/ant.ndjson",           #ant
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/apple.ndjson",        #apple
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/arm.ndjson",         #arm
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/axe.ndjson",         #axe
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/backpack.ndjson",     #backpack
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/banana.ndjson",  #banana
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/bandage.ndjson",    #bandage
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/baseball%20bat.ndjson",     #baseball bat
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/baseball.ndjson",        #baseball
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/basket.ndjson",       #basket
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/basketball.ndjson",     #basketball
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/bat.ndjson",          #bat
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/bird.ndjson",         #bird
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/bicycle.ndjson",        #bicycle
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/bathtub.ndjson",      #bathtub
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/bear.ndjson",       #bear
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/broom.ndjson",      #broom
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/bulldozer.ndjson",     #bulldozer
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/binoculars.ndjson",     #binoculars
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/bee.ndjson",            #bee
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/crown.ndjson",          #crown
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/cactus.ndjson",         #cactus
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/donut.ndjson",          #donut
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/dog.ndjson",             #dog
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/compass.ndjson",         #compass
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/cow.ndjson",             #cow
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/dragon.ndjson",           #dragon
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/carrot.ndjson",          #carrot
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/crab.ndjson",            #crab
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/church.ndjson",          #church
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/diamond.ndjson",          #diamond
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/hat.ndjson",             #hat
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/crocodile.ndjson",      #crocodile
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/eye.ndjson",           #eye
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/hexagon.ndjson",        #hexagon
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/hand.ndjson",           #hand
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/cup.ndjson",            #cup
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/hammer.ndjson",         #hammer
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/helicopter.ndjson",      #helicopter
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/bridge.ndjson",          #bridge
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/cruise%20ship.ndjson",    #cruise ship
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/guitar.ndjson",           #guitar
   "https://storage.googleapis.com/quickdraw_dataset/full/simplified/lollipop.ndjson"         #lolipop
]

# Function to create DataFrame for each URL
def create_dataframe(url, df_name):
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Split the JSON data by lines
        lines = response.content.decode('utf-8').strip().split('\n')

        # Concatenate the lines into a valid JSON format
        json_data = '[' + ','.join(lines) + ']'

        # Read the JSON data into a DataFrame
        df = pd.read_json(json_data)

        # Create an empty list to store dictionaries
        data = []

        # Iterate over each drawing
        for drawing in df['drawing']:
            # Rescale strokes coordinates
            drawing = rescale_strokes(drawing)

            # Convert strokes to pixel image
            pixel_image = strokes_to_pixels(drawing)
            flattened_pixels = pixel_image.flatten()

            # Create a list with 50 elements where the ith element is 1 and the rest are 0s
            y_list = [0] * 50
            y_list[df_name - 1] = 1

            # Append dictionary to list
            data.append({'X': flattened_pixels, 'Y': y_list})

            # Check if the number of entries is greater than or equal to 10000
            if len(data) >= 2000:
                # Create DataFrame from list of dictionaries
                globals()[f'df{df_name}'] = pd.DataFrame(data)
                # Display the DataFrame
                print(f"DataFrame df{df_name} created with {len(data)} entries.")  # Displaying the DataFrame name and the number of entries
                return
        # Create DataFrame from list of dictionaries if the loop completes without reaching 10000 entries
        globals()[f'df{df_name}'] = pd.DataFrame(data)
        # Display the DataFrame
        print(f"DataFrame df{df_name} created with {len(data)} entries.")  # Displaying the DataFrame name and the number of entries
    else:
        print(f"Error fetching data from the URL: {url}")

# Counter for DataFrame name
df_counter = 1

# Iterate over each URL
for url in urls:
    # Create DataFrame for the URL
    create_dataframe(url, df_counter)
    # Increment DataFrame counter
    df_counter += 1

DataFrame df1 created with 2000 entries.
DataFrame df2 created with 2000 entries.
DataFrame df3 created with 2000 entries.
DataFrame df4 created with 2000 entries.
DataFrame df5 created with 2000 entries.
DataFrame df6 created with 2000 entries.
DataFrame df7 created with 2000 entries.
DataFrame df8 created with 2000 entries.
DataFrame df9 created with 2000 entries.
DataFrame df10 created with 2000 entries.
DataFrame df11 created with 2000 entries.
DataFrame df12 created with 2000 entries.
DataFrame df13 created with 2000 entries.
DataFrame df14 created with 2000 entries.
DataFrame df15 created with 2000 entries.
DataFrame df16 created with 2000 entries.
DataFrame df17 created with 2000 entries.
DataFrame df18 created with 2000 entries.
DataFrame df19 created with 2000 entries.
DataFrame df20 created with 2000 entries.
DataFrame df21 created with 2000 entries.
DataFrame df22 created with 2000 entries.
DataFrame df23 created with 2000 entries.
DataFrame df24 created with 2000 entries.
D

In [ ]:
# List to store all data frames
dfs = []

# Concatenate all data frames
for i in range(1, 51):
    dfs.append(globals()[f'df{i}'])

# Concatenate all data frames into a single DataFrame
df_data = pd.concat(dfs, ignore_index=True)

# Display the concatenated DataFrame
print("Concatenated DataFrame 'df_data':")
print(df_data)

Concatenated DataFrame 'df_data':
                                                       X  \
0      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1055728221322...   
2      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1055728221322...   
3      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
...                                                  ...   
99995  [0.0, 0.0, 0.292893186843619, 1.0, 1.0, 1.0, 1...   
99996  [0.0, 0.29289320671183516, 1.0, 1.0, 1.0, 1.0,...   
99997  [0.013606106638525128, 0.17800508886543764, 0....   
99998  [0.29289320671183516, 1.0, 1.0, 1.0, 0.2928932...   
99999  [0.029857476408267192, 1.0, 0.0, 0.0, 0.0, 0.0...   

                                                       Y  
0      [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
1      [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
2      [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
3      [1

In [ ]:
X.shape


NameError: name 'X' is not defined

In [ ]:

#Forward Propagation
import numpy as np

def relu(x):
    return np.maximum(0, x)  # ReLU activation function

def softmax(x):
    exp_scores = np.exp(x - np.max(x))  # Subtract max for numerical stability
    return exp_scores / np.sum(exp_scores)

def first_layer_forward(X, W1, b1):
    Z1 = np.dot(W1, X) + b1  # Weighted sum + bias
    A1 = relu(Z1)  # Apply ReLU activation
    return A1, Z1

def second_layer_forward(A1, W2, b2):
    Z2 = np.dot(W2, A1) + b2  # Weighted sum + bias
    A2 = softmax(Z2)  # Apply softmax for probabilities
    return A2, Z2


In [ ]:
#Backward Propagation

def relu_derivative(x):
    return 1 * (x > 0)  # Derivative of ReLU

def softmax_derivative(x):
    # Compute softmax derivative only for one-hot encoded output
    s = x.reshape(-1,1)

def second_layer_backward(A1, A2, Y, Z2, W2):
    dZ2 = A2 - Y  # Softmax derivative * Cross-entropy loss derivative
    dW2 = np.dot(dZ2, A1.T)
    db2 = np.sum(dZ2, axis=1, keepdims=True)
    dA1 = np.dot(W2.T, dZ2)
    return dZ2, dW2, db2, dA1


def first_layer_backward(dA1, Z1, X):
    # A1, Z1 = first_layer_forward(X, W1, b1)
    dZ1 = dA1 * relu_derivative(Z1)  # Derivative of ReLU
    dW1 = np.dot(dZ1, X.T)
    db1 = np.sum(dZ1, axis=1, keepdims=True)
    return dZ1, dW1, db1


In [ ]:
# Function for forward propagation
def forward_propagation(X, W1, b1, W2, b2):
    A1, Z1 = first_layer_forward(X, W1, b1)
    A2, Z2 = second_layer_forward(A1, W2, b2)
    return A1, Z1, A2, Z2

# Function for backward propagation
def backward_propagation(A1, Y, Z1, A2, Z2, W1, W2,X):
    dZ2, dW2, db2, dA1 = second_layer_backward(A1, A2, Y, Z2, W2)
    dZ1, dW1, db1 = first_layer_backward(dA1, Z1, X)
    return dZ2, dW2, db2, dZ1, dW1, db1


In [ ]:
def preprocess_data(df):
    X = np.array(df['X'].tolist())
    Y = np.array(df['Y'].tolist())
    return X, Y

# Function to initialize weights and biases
def initialize_parameters(input_size, hidden_size, output_size):
    W1 = np.random.randn(hidden_size, input_size) * 0.01
    b1 = np.zeros((hidden_size, 1))
    W2 = np.random.randn(output_size, hidden_size) * 0.01
    b2 = np.zeros((output_size, 1))
    return W1, b1, W2, b2

In [ ]:
# def train_model(X_train, Y_train, hidden_size, output_size, learning_rate, num_iterations):
#     input_size = X_train.shape[1]
#     W1, b1, W2, b2 = initialize_parameters(input_size, hidden_size, output_size)

#     for i in range(len(X_train)):
#       X = X_train[i]
#       Y = Y_train[i]
#       X = X.reshape(-1, 1)  # Reshape X to match the model input
#       Y = Y.reshape(-1, 1)  # Reshape Y to match the model output
#       for i in range(num_iterations):
#         A1, Z1, A2, Z2 = forward_propagation(X, W1, b1, W2, b2)
#         dZ2, dW2, db2, dZ1, dW1, db1 = backward_propagation(A1, Y, Z1, A2, Z2, W1, W2,X)
#         # Update weights and biases
#         W1 -= learning_rate * dW1
#         b1 -= learning_rate * db1
#         W2 -= learning_rate * dW2
#         b2 -= learning_rate * db2

#         if i % 100 == 0:
#               print(f'Iteration {i}: Loss {np.mean(-np.sum(Y * np.log(A2.T), axis=0))}')


#     return W1, b1, W2, b2

def train_model(X_train, Y_train, hidden_size, output_size, learning_rate, num_iterations):
    input_size = X_train.shape[1]
    W1, b1, W2, b2 = initialize_parameters(input_size, hidden_size, output_size)

    for iteration in range(num_iterations):
        total_loss = 0
        for X, Y in zip(X_train, Y_train):
            X = X.reshape(-1, 1)  # Reshape X to match the model input
            Y = Y.reshape(-1, 1)  # Reshape Y to match the model output
            A1, Z1, A2, Z2 = forward_propagation(X, W1, b1, W2, b2)

            # Calculate loss
            loss = np.mean(-np.sum(Y * np.log(A2.T), axis=1))
            total_loss += loss

            dZ2, dW2, db2, dZ1, dW1, db1 = backward_propagation(A1, Y, Z1, A2, Z2, W1, W2, X)

            # Update weights and biases
            W1 -= learning_rate * dW1
            b1 -= learning_rate * db1
            W2 -= learning_rate * dW2
            b2 -= learning_rate * db2

        # Print average loss for the iteration
        average_loss = total_loss / len(X_train)
        if iteration % 100 == 0:
            print(f'Iteration {iteration}: Loss {average_loss}')

    return W1, b1, W2, b2


In [ ]:
# def test_model(X_test, Y_test, W1, b1, W2, b2):
#     for X, Y in zip(X_test, Y_test):
#       X = X.reshape(-1, 1)  # Reshape X to match the model input
#       Y = Y.reshape(-1, 1)  # Reshape Y to match the model output
#       A1, _, A2, _ = forward_propagation(X, W1, b1, W2, b2)
#       predictions = np.argmax(A2.T, axis=1)
#       true_labels = np.argmax(Y, axis=1)
#       accuracy = np.mean(predictions == true_labels)
#       return accuracy

def test_model(X_test, Y_test, W1, b1, W2, b2):
    accuracies = []  # List to store accuracies for each example

    for X, Y in zip(X_test, Y_test):
        X = X.reshape(-1, 1)  # Reshape X to match the model input
        Y = Y.reshape(-1, 1)  # Reshape Y to match the model output

        # Forward propagation to get predictions
        A1, _, A2, _ = forward_propagation(X, W1, b1, W2, b2)

        # Calculate predictions and true labels
        predictions = np.argmax(A2.T, axis=1)
        true_labels = np.argmax(Y, axis=0)

        # Calculate accuracy for this example
        accuracy = np.mean(predictions == true_labels)
        accuracies.append(accuracy)

    # Calculate overall accuracy as the mean of accuracies for all examples
    overall_accuracy = np.mean(accuracies)
    return overall_accuracy


In [ ]:
from sklearn.model_selection import train_test_split

X, Y = preprocess_data(df_data)
# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(X_train.shape)
# Define hyperparameters
hidden_size = 256
output_size = 50
learning_rate = 0.01
num_iterations = 1


# Train the model
W1, b1, W2, b2 = train_model(X_train, Y_train, hidden_size, output_size, learning_rate, num_iterations)



(80000, 784)
Iteration 0: Loss 6.238957460016874


ValueError: shapes (256,784) and (20000,784) not aligned: 784 (dim 1) != 20000 (dim 0)

In [ ]:
# Test the model
accuracy = test_model(X_test, Y_test, W1, b1, W2, b2)
print(f'Test Accuracy: {accuracy}')

Test Accuracy: 0.49905


In [ ]:
print(W1, b1, W2, b2)

[[-0.11137663 -0.28565503  0.18028531 ...  0.01265253  0.01215225
  -0.00646696]
 [ 0.09737304  0.38403556  0.40655926 ...  0.00899322  0.00551921
  -0.00239803]
 [-0.10841771  0.07416826  0.37971861 ...  0.0140245  -0.00694985
  -0.00091622]
 ...
 [-0.13647365  0.14139149  0.02754494 ...  0.00473847  0.01875681
   0.00619087]
 [-0.28333617 -0.4766045   0.04873217 ... -0.00471359  0.00245239
   0.00423721]
 [-0.18742801  0.09101404  0.04649445 ...  0.0050699   0.01243972
   0.00709027]] [[ 4.27330959e-01]
 [-2.50344329e-01]
 [-9.72166636e-01]
 [-1.71608310e+00]
 [-3.38007428e-01]
 [-4.85316792e+00]
 [-1.33196591e-01]
 [-1.05047762e+00]
 [ 1.37685188e-01]
 [-7.18308855e-01]
 [ 9.99753541e-01]
 [ 1.04704563e+00]
 [ 9.21739337e-01]
 [-3.42894442e-02]
 [ 5.36982614e-01]
 [ 2.67292723e-01]
 [ 4.90076507e-01]
 [ 2.76498189e-03]
 [ 1.01667140e+00]
 [-4.71000337e-02]
 [-3.50632436e-01]
 [ 1.14997896e+00]
 [-1.07608039e-01]
 [-6.85150730e-01]
 [-2.94325002e+00]
 [ 1.36335602e+00]
 [ 3.75746378e

In [ ]:
Xnew = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.29289321881345254, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.29289321881345254, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.29289321881345254, 1.0, 0.29289321881345254, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.29289321881345254, 1.0, 0.29289321881345254, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.29289321881345254, 1.0, 0.29289321881345254, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.29289321881345254, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.29289321881345254, 1.0, 0.29289321881345254, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.29289321881345254, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.29289321881345254, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.29289321881345254, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.29289321881345254, 0.0, 0.29289321881345254, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.29289321881345254, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.29289321881345254, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.29289321881345254, 0.0, 0.0, 0.29289321881345254, 1.0, 1.0, 1.0, 1.0, 0.29289321881345254, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.29289321881345254, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.29289321881345254, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]).reshape(-1,1)

In [ ]:
A1, Z1, A2, Z2 = forward_propagation(Xnew, W1, b1, W2, b2)
print (A2)

[[2.63726365e-14]
 [1.08007116e-15]
 [5.44626543e-22]
 [1.27635335e-10]
 [7.28780805e-11]
 [3.99657947e-11]
 [3.58109330e-09]
 [4.85997219e-10]
 [5.66951662e-08]
 [9.91773177e-01]
 [8.38568166e-16]
 [7.14162671e-10]
 [4.59551480e-06]
 [1.01798976e-14]
 [1.20715220e-14]
 [4.79973945e-07]
 [1.06806592e-13]
 [1.59426502e-10]
 [4.52552385e-10]
 [1.66831563e-12]
 [1.25491296e-09]
 [4.66291343e-09]
 [7.51548656e-09]
 [7.43537132e-12]
 [3.68763688e-16]
 [2.61714435e-14]
 [1.21230169e-19]
 [2.83063459e-20]
 [3.44702793e-11]
 [1.57312619e-23]
 [1.69639614e-04]
 [1.70518815e-14]
 [2.25058853e-10]
 [4.33641021e-11]
 [4.08703717e-14]
 [1.28730379e-15]
 [1.70279248e-06]
 [2.09115275e-12]
 [1.08741789e-15]
 [9.42393059e-11]
 [4.26073727e-20]
 [8.04713888e-03]
 [1.10827257e-06]
 [1.75647382e-06]
 [7.67495188e-12]
 [1.31250278e-15]
 [6.15737404e-10]
 [2.67892558e-13]
 [1.16440957e-19]
 [3.24835872e-07]]
